# Dependency Error = Version compatibility problem



=== magenta/magenta/version.py ===

__version__ = '2.1.4'

In [2]:
# 셀 실행 후 런타임 다시 시작 
!pip install magenta==2.1.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Preprocessing

## Import library

In [1]:
import tensorflow as tf
import numpy as np
import zipfile 
import os
import pandas as pd
import IPython
import collections
import note_seq 

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data
from magenta.scripts.convert_dir_to_note_sequences import convert_directory , convert_midi 
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import tensorflow.compat.v1 as tf
import tf_slim 


/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

# Make dataset 

Download data

In [3]:
!gdown 1TVJgOECJ-f_a1AK8HNfB9GDXxY0ssQcr

Downloading...
From: https://drive.google.com/uc?id=1TVJgOECJ-f_a1AK8HNfB9GDXxY0ssQcr
To: /content/groove-v1.0.0-midionly.zip
100% 3.26M/3.26M [00:00<00:00, 214MB/s]


In [4]:
# Make directory
!mkdir data_dir

mkdir: cannot create directory ‘data_dir’: File exists


In [5]:
# zip파일 압축해제 
with zipfile.ZipFile('./groove-v1.0.0-midionly.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data_dir')

In [7]:
#경로지정 - convert_directory

# 압축해제된 데이터 경로 
root_dir = '/content/data_dir/groove' 
# MIDI 파일을 TFrecord로 변환한 파일 경로 
output_file = '/content/data_dir/sequence.tfrecord'


In [8]:
# MIDI파일 정보 확인 
df = pd.read_csv('/content/data_dir/groove/info.csv')
df = pd.DataFrame(df)
df.head(10)

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
5,drummer1,drummer1/eval_session,drummer1/eval_session/5,funk/groove5,84,beat,4-4,drummer1/eval_session/5_funk-groove5_84_beat_4...,drummer1/eval_session/5_funk-groove5_84_beat_4...,45.687518,test
6,drummer1,drummer1/eval_session,drummer1/eval_session/6,hiphop/groove6,87,beat,4-4,drummer1/eval_session/6_hiphop-groove6_87_beat...,drummer1/eval_session/6_hiphop-groove6_87_beat...,44.119242,test
7,drummer1,drummer1/eval_session,drummer1/eval_session/7,pop/groove7,138,beat,4-4,drummer1/eval_session/7_pop-groove7_138_beat_4...,drummer1/eval_session/7_pop-groove7_138_beat_4...,27.706547,test
8,drummer1,drummer1/eval_session,drummer1/eval_session/8,rock/groove8,65,beat,4-4,drummer1/eval_session/8_rock-groove8_65_beat_4...,drummer1/eval_session/8_rock-groove8_65_beat_4...,59.067313,test
9,drummer1,drummer1/eval_session,drummer1/eval_session/9,soul/groove9,105,beat,4-4,drummer1/eval_session/9_soul-groove9_105_beat_...,drummer1/eval_session/9_soul-groove9_105_beat_...,36.540504,test


In [9]:
# 총 갯수 
len(df)

1150

## Convert MIDI Directory to TFrecord 

In [10]:
# == magenta/scripts/convert_dir_to_note_sequences.py == 
# == convert_directory 사용 == 디렉토리 전체를 변환해주는 함수 

convert_directory(root_dir,output_file,recursive=True) 

# Config

In [11]:
# == magenta/models/music_vae/configs.py == 


class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)


CONFIG_MAP = {}


HParams = contrib_training.HParams

# GrooVAE configs 중 4마디를 추출하기 위해서 'groovae_4bar'config를 차용하여 논문에 사용했던 모델과 비슷하게 적용

CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(), # BidirectionalLstmEncoder
                   lstm_models.GrooveLstmDecoder()), # Groove LSTM Decoder with MSE loss for continuous values
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512, # Minibatch size
            max_seq_len= 16 * 4,  # 4 bars with 16 steps per bar 
            z_size= 256, # Size of latent vector  
            enc_rnn_size=[512, 512], # Size of Encoder => 논문에서는 16마디의 데이터셋으로 [2048,2048] 하여 1/4인 [512,512] 적용
            dec_rnn_size=[256, 256],# Size of decoder  => 논문에서는 16마디의 데이터셋으로 [1024,1024] 하여 1/4인 [256,256] 적용
            max_beta=0.2, # Maximum KL cost weight, or cost if not annealing
            free_bits= 48, # Bits to exclude from KL loss per dimension
            dropout_keep_prob=0.3, 
            learning_rate = 0.001, # Learning rate                       ==> 논문에서 사용
            decay_rata = 0.9999, # Learning rate decay per minibatch     ==> 논문에서 사용
            min_learning_rate = 0.00001 ,                             #   ==> 논문에서 사용 
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,

        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # train data 경로 설정 
    train_examples_path='/content/data_dir/sequence.tfrecord',
)

# Train 

In [12]:
#============ License=====================================================
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#=======================================================================

#=== MusicVAE train script===
#== magenta/models/music_vae/music_vae_train.py ==

# Should not be called from within the graph to avoid redundant summaries.
def _trial_summary(hparams, examples_path, output_dir):
     #=== Writes a tensorboard text summary of the trial===

    examples_path_summary = tf.summary.text(
        'examples_path', tf.constant(examples_path, name='examples_path'),
        collections=[])

    hparams_dict = hparams.values()

#===  Create a markdown table from hparams ===
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
        'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()


def _get_input_tensors(dataset, config):
    #== Get input tensors from dataset ===
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,sequence_length) = iterator.get_next()
    input_sequence.set_shape(
        [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
        [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth])
    sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())
        
    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

#=== Set Training checkpoints and hours ===
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep= 5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):

#==== Train loop ====
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
                num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            #== Optimizer ===
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')

            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)


def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
  # Load model params, save config file and start trainer
    config = config_map['groovae_4bar']
    train_dir = '/content/train'
    num_steps = 50000  #epoch
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            checkpoints_to_keep = 5,
            keep_checkpoint_every_n_hours = 1,
            num_steps=num_steps)      
  
    else:
        print("Training is False")

In [13]:
run(CONFIG_MAP) #epoch 50000

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/usr/lo

# Generate

In [14]:
!mkdir generated_midi

mkdir: cannot create directory ‘generated_midi’: File exists


## Batch size별 generated_midi 

512

In [29]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===
model = TrainedModel(
    config=configs.CONFIG_MAP['groovae_4bar'],
    batch_size = 512, # min(max_batch_size, num_outputs)
    checkpoint_dir_or_path='/content/train') # 체크포인트의 경로

drum_samples = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(drum_samples[0], '/content/generated_midi/drum_4bar_512.mid')

/usr/local/lib/python3.8/dist-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/usr/local/lib/python3.8/dist-packages/magenta

256

In [24]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===
model = TrainedModel(
    config=configs.CONFIG_MAP['groovae_4bar'],
    batch_size = 256, # min(max_batch_size, num_outputs)
    checkpoint_dir_or_path='/content/train') # 체크포인트의 경로

drum_samples = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(drum_samples[0], '/content/generated_midi/drum_4bar_256.mid')

128

In [23]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===
model = TrainedModel(
    config=configs.CONFIG_MAP['groovae_4bar'],
    batch_size = 128, # min(max_batch_size, num_outputs)
    checkpoint_dir_or_path='/content/train') # 체크포인트의 경로

drum_samples = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(drum_samples[0], '/content/generated_midi/drum_4bar_128.mid')

1

In [16]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===
model = TrainedModel(
    config=configs.CONFIG_MAP['groovae_4bar'],
    batch_size = 1, # min(max_batch_size, num_outputs)
    checkpoint_dir_or_path='/content/train') # 체크포인트의 경로

drum_samples = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(drum_samples[0], '/content/generated_midi/drum_4bar_1.mid')

In [28]:
drum_samples[0]

tempos {
  qpm: 120.0
}
notes {
  pitch: 36
  velocity: 76
  start_time: -0.013642675243318081
  end_time: 0.11135732475668192
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 112
  start_time: -0.012215925380587578
  end_time: 0.11278407461941242
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 26
  start_time: -0.0149957574903965
  end_time: 0.1100042425096035
  instrument: 9
  is_drum: true
}
notes {
  pitch: 51
  velocity: 91
  start_time: -0.01838684268295765
  end_time: 0.10661315731704235
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 70
  start_time: 0.2362657617777586
  end_time: 0.3612657617777586
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 67
  start_time: 0.22768578492105007
  end_time: 0.35268578492105007
  instrument: 9
  is_drum: true
}
notes {
  pitch: 38
  velocity: 7
  start_time: 0.3631692286580801
  end_time: 0.4881692286580801
  instrument: 9
  is_drum: true
}
notes {
  pitch: 51
  velocity: